In [1]:
import os
for dirname, _, filenames in os.walk('C:\\Users\\Java\\Diplomski- movies\\data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Users\Java\Diplomski- movies\data\credits.csv
C:\Users\Java\Diplomski- movies\data\keywords.csv
C:\Users\Java\Diplomski- movies\data\links.csv
C:\Users\Java\Diplomski- movies\data\links_small.csv
C:\Users\Java\Diplomski- movies\data\movies_metadata.csv
C:\Users\Java\Diplomski- movies\data\ratings.csv
C:\Users\Java\Diplomski- movies\data\ratings_small.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD, NMF
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate
from surprise import accuracy
import surprise as surprise

Ucitavanje metapodataka

In [3]:
meta = pd.read_csv('data/movies_metadata.csv')
meta.head()

C:\Users\Java\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Ucitavanje ocena za manji set

In [4]:
ratings = pd.read_csv('data/ratings_small.csv')
ratings.head() 

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
links = pd.read_csv('data/links_small.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
keywords = pd.read_csv('data/keywords.csv')
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


Ucitavanje informacija o glumcima/postavi

In [7]:
credits = pd.read_csv('data/credits.csv')
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Fokusirano na sadržaj- popunjavanje vrednosti

In [8]:
meta['overview'] = meta['overview'].fillna('')

Prikaz formata opisa filmova

In [9]:
meta['overview'].head() 

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [10]:
# provera tipova podataka
pd.DataFrame({'feature':meta.dtypes.index, 'dtype':meta.dtypes.values})

,feature,dtype
0,adult,object
1,belongs_to_collection,object
2,budget,object
3,genres,object
4,homepage,object
5,id,object
6,imdb_id,object
7,original_language,object
8,original_title,object
9,overview,object


Izbacivanje redova koji prave problem

In [11]:
meta = meta.drop([19730, 29503, 35587]) 


meta['id'] = pd.to_numeric(meta['id'])


Provera tipova podataka

In [12]:
pd.DataFrame({'feature':links.dtypes.index, 'dtype':links.dtypes.values})

,feature,dtype
0,movieId,int64
1,imdbId,int64
2,tmdbId,float64


Popravljanje tipova podataka

In [13]:
col=np.array(links['tmdbId'], np.int64)
links['tmdbId']=col

Spajanje podataka 

In [14]:
meta.rename(columns={'id':'tmdbId'}, inplace=True)
meta = pd.merge(meta,links,on='tmdbId')
meta.drop(['imdb_id'], axis=1, inplace=True)
meta.head()

# manji
#meta = meta[meta['tmdbId'].isin(links)]
#meta.shape

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,imdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1,114709
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,2,113497
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,3,113228
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,4,114885
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,5,113041


Uklanjanje reči koje smetaju i napraviti TF-IDF vektor matricu

In [15]:
# Sklanjanje reči
tfidf = TfidfVectorizer(stop_words='english')
# matrix
tfidf_matrix = tfidf.fit_transform(meta['overview'])
tfidf_matrix.shape

(9099, 29727)

Kosinus sličnost i odgovarajući filmovi u skladu

In [16]:
#Kosinus slicnost
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# Filmovi bez duplikata
indices = pd.Series(meta.index, index=meta['original_title']).drop_duplicates()

Pravljenje funkcije za preporucivanje

In [39]:

def get_recommendations(title, cosine_sim=cosine_sim):
    
    #uzeti naslov filma
    idx = indices[title]
    # trazimo slicne filmove na osnovu kosinus slicnosti
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sortiranje filmova na osnovu slicnosti skorova
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # 10 prvih filmova po slicnosti 
    sim_scores = sim_scores[1:11]
    # uzeti filmove
    movie_indices = [i[0] for i in sim_scores]
    # skloniti outliere i nisko ocenjenje filmova
    for i in movie_indices:
        pop = meta.at[i,'vote_average']
        if pop<5 or pop>10:
            movie_indices.remove(i)

    # vratiti slicne a da je treshold postavljen na 5
    return meta[['original_title','vote_average']].iloc[movie_indices]

In [40]:
get_recommendations('Inception')

,original_title,vote_average
1705,House,5.8
2830,What Ever Happened to Baby Jane?,7.9
8577,Mission: Impossible - Rogue Nation,7.1
5247,Cypher,6.7
319,Cobb,6.1
141,Crumb,7.6
8302,Syrup,5.7
1952,At First Sight,5.9
9077,Central Intelligence,6.2
8873,Pitch Perfect 2,6.8


In [41]:
get_recommendations('The Conjuring')

,original_title,vote_average
9074,The Conjuring 2,7.0
5587,The Boston Strangler,6.7
8726,The Borderlands,5.1
3077,The Spiral Staircase,6.6
5790,The Turning Point,6.0
8139,Sinister,6.8
3784,Things Behind the Sun,5.9
353,Jason's Lyric,5.9
6118,The Amityville Horror,6.0


# Algoritmi

In [20]:
# algoritam za korisnike

reader = Reader() # Used to parse a file containing ratings
df = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(df) # Split the data into folds

<generator object KFold.split at 0x0000017620D471C8>

In [21]:
# svd, kros validacija i treniranje
svd = SVD()
cross_validate(svd, df, measures=['RMSE', 'MAE'])

trainset = df.build_full_trainset()
svd.fit(trainset)


isto za NMF

In [22]:
nmf = NMF()
cross_validate(nmf, df, measures=['RMSE', 'MAE'])

trainset = df.build_full_trainset()
svd.fit(trainset)

In [43]:
# Izabrati korisnika
ratings[ratings['userId'] == 55]

,userId,movieId,rating,timestamp
8222,55,1,3.0,855926941
8223,55,5,3.0,855926988
8224,55,6,5.0,855926988
8225,55,7,2.0,855926988
8226,55,9,3.0,855927315
8227,55,32,4.0,855926941
8228,55,65,5.0,855927315
8229,55,74,3.0,855927438
8230,55,79,3.0,855927172
8231,55,95,3.0,855926941


In [44]:
# Citanje manjih fajlova
links_df = pd.read_csv('data/links_small.csv')
col=np.array(links_df['tmdbId'], np.int64)
links_df['tmdbId']=col

# Spajanje matapodataka i linkova manjeg seta podataka
links_df = links_df.merge(meta[['title', 'tmdbId']], on='tmdbId').set_index('title')
links_index = links_df.set_index('tmdbId') # za ozanke 

# Hibrdini model - funkcija preporuke

Prvi koji funkcionise na sa SVD algoritmom

In [49]:

def hybrid(userId, title):
    idx = indices[title]
    tmdbId = links_df.loc[title]['tmdbId'] #spojiti preko odgovarajućeg id-a
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21] # Skor 20 najsličnijih filmova
    movie_indices = [i[0] for i in sim_scores]
    
    movies = meta.iloc[movie_indices][['title', 'vote_average', 'tmdbId']]
    #procena uz pomoc SVD algoritma
    movies['est'] = movies['tmdbId'].apply(lambda x: svd.predict(userId, links_index.loc[x]['movieId']).est) 
    
    

    movies = movies.sort_values('est', ascending=False) # Rangirati po procenama
    
    movies.columns = ['Title', 'Vote Average', 'TMDb Id', 'Estimated Prediction']
    return movies.head(10) #prikazi 1o 

Drugi koji fukncionise sa NFM algoritmom

In [52]:
# Recommendation function
def hybrid2(userId, title):
    idx = indices[title]
    tmdbId = links_df.loc[title]['tmdbId'] 
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21] 
    movie_indices = [i[0] for i in sim_scores]
    
    movies = meta.iloc[movie_indices][['title', 'vote_average', 'tmdbId']]
    movies['est'] = movies['tmdbId'].apply(lambda x: nmf.predict(userId, links_index.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False) 
    
    
    movies.columns = ['Title', 'Vote Average', 'TMDb Id', 'Estimated Prediction']
    return movies.head(10) 

In [53]:
#izabrati korisnika
hybrid(78, 'Midnight in Paris')


,Title,Vote Average,TMDb Id,Estimated Prediction
856,Everyone Says I Love You,6.5,9716,4.697548
4502,The Life of David Gale,7.3,11615,4.574624
362,The Age of Innocence,7.0,10436,4.565999
2340,Children of Paradise,7.9,2457,4.513485
4261,Punch-Drunk Love,6.9,8051,4.456575
4008,I Think I Do,4.7,19797,4.423691
8111,Conception,4.4,83403,4.301803
4420,Nicholas Nickleby,6.8,29339,4.300775
5790,The Turning Point,6.0,61280,4.281214
5226,Zorba the Greek,7.0,10604,4.233172


In [54]:
hybrid2(78, 'Midnight in Paris')

,Title,Vote Average,TMDb Id,Estimated Prediction
2340,Children of Paradise,7.9,2457,5.000000
5226,Zorba the Greek,7.0,10604,5.000000
4261,Punch-Drunk Love,6.9,8051,5.000000
8933,Hands in the Air,6.0,54155,5.000000
4502,The Life of David Gale,7.3,11615,4.858316
3784,Things Behind the Sun,5.9,102933,4.439436
4008,I Think I Do,4.7,19797,4.426141
4438,A Little Romance,7.5,20813,4.392028
856,Everyone Says I Love You,6.5,9716,4.122034
362,The Age of Innocence,7.0,10436,4.080506


In [55]:
# Recommendations for user with id 30
hybrid(44, 'Inception')

,Title,Vote Average,TMDb Id,Estimated Prediction
1045,Being There,7.7,10322,3.757300
6232,A History of Violence,6.9,59,3.756025
5247,Cypher,6.7,10133,3.704274
2830,What Ever Happened to Baby Jane?,7.9,10242,3.536267
9077,Central Intelligence,6.2,302699,3.488465
141,Crumb,7.6,26564,3.464474
319,Cobb,6.1,29973,3.445341
8577,Mission: Impossible - Rogue Nation,7.1,177677,3.442218
6680,Benny's Video,7.1,1987,3.415933
6052,The Seven-Per-Cent Solution,7.1,29578,3.357406


In [56]:
hybrid2(44, 'Inception')

,Title,Vote Average,TMDb Id,Estimated Prediction
5247,Cypher,6.7,10133,4.756975
141,Crumb,7.6,26564,3.940170
6680,Benny's Video,7.1,1987,3.850652
2830,What Ever Happened to Baby Jane?,7.9,10242,3.673032
1705,House,5.8,11415,3.672378
8577,Mission: Impossible - Rogue Nation,7.1,177677,3.615920
1045,Being There,7.7,10322,3.581734
8596,They Came Together,5.3,180296,3.543979
6232,A History of Violence,6.9,59,3.480239
4822,Where Eagles Dare,7.4,11046,3.362272


In [57]:

hybrid(500, 'Happiness')

,Title,Vote Average,TMDb Id,Estimated Prediction
6245,Angels in America,6.8,24019,3.677633
7735,Emma,7.6,183894,3.672483
5176,Kwaidan,7.5,30959,3.410363
3514,More,7.9,36107,3.327150
7835,One Day,7.0,51828,3.246415
4111,Thirteen Conversations About One Thing,6.4,17734,3.227723
6783,The Seeker: The Dark Is Rising,4.8,2274,3.217882
1861,Nights of Cabiria,7.9,19426,3.217710
7175,Dr. Horrible's Sing-Along Blog,7.8,14301,3.196929
5057,Dr. Jekyll and Mr. Hyde,6.6,3022,3.069093


In [58]:
hybrid2(500, 'Happiness')

,Title,Vote Average,TMDb Id,Estimated Prediction
6245,Angels in America,6.8,24019,3.953002
5205,The China Syndrome,7.0,988,3.716693
5057,Dr. Jekyll and Mr. Hyde,6.6,3022,3.596294
5176,Kwaidan,7.5,30959,3.543979
4111,Thirteen Conversations About One Thing,6.4,17734,3.535461
8169,Excision,5.9,84194,3.488424
7331,The Cove,7.9,23128,3.480188
1861,Nights of Cabiria,7.9,19426,3.201231
7735,Emma,7.6,183894,3.100357
7175,Dr. Horrible's Sing-Along Blog,7.8,14301,3.028168
